# Imports

In [1]:
# standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# magic lines
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import re

# Load data

In [3]:
! ls ../data

all_games_2023-01_to_2024-06_all_cols.csv
all_games_2023-01_to_2024-06_raw.csv
all_games_2024-04_to_2024-06_all_cols.csv
all_games_2024-04_to_2024-06_raw.csv
select_games_2023-01_to_2024-06_select_cols.csv
select_games_2024-04_to_2024-06_select_cols.csv


In [4]:
# filename = "all_games_df_2023-01_to_2024-05.csv"
filename = "all_games_2023-01_to_2024-06_all_cols.csv"
filepath = os.path.join("..","data", filename)
all_games_df_raw = pd.read_csv(filepath)

In [5]:
all_games_df_raw

,date,url,start_time,end_time,time_class,time_control,white_username,black_username,user_colour,result,...,pawns_count_diff.1,winner_material_ttl,winner_material_ttl.1,loser_material_ttl,loser_material_ttl.1,winner_pcs,winner_pcs.1,loser_pcs,loser_pcs.1,user_win
0,2023-01-30,https://www.chess.com/game/live/68853253455,2023-01-30 21:29:25,2023-01-30 21:39:52,blitz,300,ivangard57,JammyNinja,black,1-0,...,0,9,9,0,0,"['K', 'Q']","['K', 'Q']",['k'],['k'],False
1,2023-03-15,https://www.chess.com/game/live/72649318887,2023-03-15 19:56:11,2023-03-15 20:02:18,blitz,300,axdasilva129,JammyNinja,black,0-1,...,3,12,12,1,1,"['p', 'p', 'p', 'p', 'n', 'k', 'r']","['p', 'p', 'p', 'p', 'n', 'k', 'r']","['P', 'K']","['P', 'K']",True
2,2023-03-25,https://www.chess.com/game/live/73488582235,2023-03-25 13:03:42,2023-03-25 13:11:44,blitz,300,Bhashyam2K23,JammyNinja,black,1-0,...,3,19,19,7,7,"['P', 'P', 'Q', 'P', 'P', 'P', 'R', 'K']","['P', 'P', 'Q', 'P', 'P', 'P', 'R', 'K']","['k', 'p', 'p', 'r']","['k', 'p', 'p', 'r']",False
3,2023-03-29,https://www.chess.com/game/live/73819849367,2023-03-29 09:06:37,2023-03-29 09:12:33,blitz,300,JammyNinja,callmedaddycool,white,1/2-1/2,...,0,25,25,0,0,"['r', 'r', 'n', 'k', 'n', 'n', 'n', 'b']","['r', 'r', 'n', 'k', 'n', 'n', 'n', 'b']",['K'],['K'],False
4,2023-04-04,https://www.chess.com/game/live/74361801303,2023-04-04 15:38:52,2023-04-04 16:05:50,rapid,600+5,JammyNinja,danderosalia,white,1-0,...,-2,16,16,9,9,"['R', 'Q', 'P', 'K', 'P']","['R', 'Q', 'P', 'K', 'P']","['k', 'p', 'p', 'p', 'p', 'r']","['k', 'p', 'p', 'p', 'p', 'r']",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3099,2024-05-31,https://www.chess.com/game/live/110898007115,2024-05-31 12:33:22,2024-05-31 12:35:40,bullet,60,Ingbing,JammyNinja,black,1-0,...,2,22,22,0,0,"['R', 'Q', 'N', 'P', 'P', 'K', 'N']","['R', 'Q', 'N', 'P', 'P', 'K', 'N']",['k'],['k'],False
3100,2024-05-31,https://www.chess.com/game/live/110898034915,2024-05-31 12:35:50,2024-05-31 12:37:37,bullet,60,JammyNinja,nhathuy82vn,white,1-0,...,1,31,31,30,30,"['R', 'P', 'P', 'N', 'Q', 'P', 'B', 'P', 'P', ...","['R', 'P', 'P', 'N', 'Q', 'P', 'B', 'P', 'P', ...","['r', 'q', 'r', 'k', 'p', 'b', 'p', 'p', 'p', ...","['r', 'q', 'r', 'k', 'p', 'b', 'p', 'p', 'p', ...",True
3101,2024-05-31,https://www.chess.com/game/live/110898057561,2024-05-31 12:37:42,2024-05-31 12:39:28,bullet,60,JammyNinja,CalculusV,white,0-1,...,0,19,19,15,15,"['r', 'k', 'p', 'p', 'p', 'p', 'p', 'q']","['r', 'k', 'p', 'p', 'p', 'p', 'p', 'q']","['P', 'P', 'P', 'P', 'P', 'R', 'R', 'K']","['P', 'P', 'P', 'P', 'P', 'R', 'R', 'K']",False
3102,2024-05-31,https://www.chess.com/game/live/110902287015,2024-05-31 13:49:35,2024-05-31 13:51:50,bullet,60,Competent_Checkmate,JammyNinja,black,1-0,...,0,21,21,4,4,"['Q', 'R', 'P', 'P', 'P', 'N', 'P', 'K']","['Q', 'R', 'P', 'P', 'P', 'N', 'P', 'K']","['k', 'p', 'p', 'p', 'p']","['k', 'p', 'p', 'p', 'p']",False


In [7]:
test_game = all_games_df_raw.query("time_class == 'bullet'").iloc[1]
# test_moves = test_game["moves"]
test_game

date                                                           2023-10-24
url                           https://www.chess.com/game/live/91903723131
start_time                                            2023-10-24 16:22:59
end_time                                              2023-10-24 16:25:57
time_class                                                         bullet
time_control                                                         60+1
white_username                                                 JammyNinja
black_username                                                    Stief14
user_colour                                                         white
result                                                                1-0
winner                                                              white
higher_rated_colour                                                 white
white_rating                                                          824
black_rating                          

# Add new columns

In [8]:
def process_moves(row):
    """
        Looks at 'moves' column which is pgn string, extracts each players moves and when they castled
        ['move_numbers', 'white_moves', 'white_clock', 'black_moves', 'black_clock', 
        'w_cstl_side', 'w_cstl_move', 'b_cstl_side', 'b_cstl_move']
    """
    #.apply() complained that it was float when didn't manually convert to string?
    moves = str(row["moves"])
    moves = moves.replace("{","")

    move_nums = moves.split()[::8]
    white_moves = moves.split()[1::8]
    black_moves = moves.split()[5::8]      
    
    white_clock = [m[:-1] for m in moves.split()[3::8] ]
    black_clock = [m[:-1] for m in moves.split()[7::8] ]
     
    if len(black_moves) < len(white_moves):
        black_moves.append(np.nan)
        black_clock.append(np.nan)
    
    #deal with castling
    w_castled, w_castled_side = 0,np.nan
    b_castled, b_castled_side = 0, np.nan
    
    for move_num, w_move, b_move in zip(move_nums, white_moves, black_moves):
        move_num = int(move_num[:-1]) #7. -> 7
        
        if w_move == "O-O":
            w_castled_side = "King"
            w_castled = move_num
        elif w_move == "O-O-O":
            w_castled_side = "Queen"
            w_castled = move_num
            
        if b_move == "O-O":
            b_castled_side = "King"
            b_castled = move_num
        elif b_move == "O-O-O":
            b_castled_side = "Queen"
            b_castled = move_num
            
#     print(f"white castled {w_castled_side} side on move {w_castled}")
#     print(f"black castled {b_castled_side} side on move {b_castled}")
    
    df_dict = {
        "move_numbers" : move_nums,
        "white_moves"  : white_moves,
        "white_clock" : white_clock,
        "black_moves"  : black_moves,
        "black_clock" : black_clock,
        
        "w_cstl_side" : w_castled_side,
        "w_cstl_move" : w_castled,        
        "b_cstl_side" : b_castled_side,
        "b_cstl_move" : b_castled
    }
    
    return df_dict
    
process_moves(all_games_df_raw.iloc[0])

KeyError: 'moves'

In [ ]:
def process_winner_rating(row):
    
    winner = "white" if row["result"] == "1-0" else "black" if row["result"] == "0-1" else "DRAW"
    
    w_rating, b_rating = row["white_rating"], row["black_rating"] #reduce line size
    
    higher_rated = "white" if w_rating > b_rating else "black" #assumes draw is good for black
    
    user_colour = "white" if row["white_username"] == "JammyNinja" else "black"
        
    def did_user_win(user = "JammyNinja"):
        if winner == "white" and row["white_username"] == user:
            return True
        if winner == "black" and row["black_username"] == user:
            return True
        if winner == "draw":
            return "DRAW"
        return False
    
    user_win = did_user_win()
    
    user_rating = w_rating if user_colour == "white" else b_rating
    
    winner_rating_diff = w_rating - b_rating if winner == "white" else b_rating - w_rating
    user_rating_diff = w_rating - b_rating if user_colour == "white" else b_rating - w_rating
        
    return {
        "higher_rated_colour" : higher_rated,
        "winner"       : winner,
        "winner_rating_diff"  : winner_rating_diff,
        "user_rating_diff" : user_rating_diff,
        "user_rating" : user_rating,
        "user_win"     : user_win,
        "user_colour" : user_colour
    }
process_winner_rating(all_games_df_raw.iloc[4]).keys()

In [ ]:
def user_castled(row):
#         print(row)
        user_castled = False
        opp_castled = False

        w_cstl_move = row["w_cstl_move"]
        b_cstl_move = row["b_cstl_move"]

        if row["user_colour"] == "white":
            if w_cstl_move > 0:
                user_castled = True
                user_cstl_diff = w_cstl_move - b_cstl_move  
            if b_cstl_move > 0:
                opp_castled = True
        else: #user is black
            if b_cstl_move > 0:
                user_castled = True
                user_cstl_diff = b_cstl_move - w_cstl_move 
            if w_cstl_move > 0:
                opp_castled = True
                
        
        #create columns to do with who castled first         
        #null if user didn't castle
        if not user_castled:
            user_cstl_diff = np.nan
        #WHAT IF I CASTLE AND THEY DONT - cant subtract?
        
        return {
            "user_castled" : user_castled,
            "opp_castled" : opp_castled,
#             "user_cstl_diff" : user_cstl_diff,
#             "winner_cstl_diff" : win_cstl_diff
        }
# user_castled(all_games_df.iloc[80])
# display(all_games_df.iloc[80])

In [ ]:
test_game

In [ ]:
def pieces_final_position(game):
    """
    starts with eighth rank, first file to eighth
    white is uppercase, black lowercase
    p: pawn, r: rook, n: knight, b: bishop, q : queen, k : king 
    """
    fen = game["final_position_fen"]
    fin_pos = fen.split()[0]
    
    board = fin_pos.split("/")
    ttl_white_pcs, ttl_black_pcs = 0,0
    
    #white pieces are uppercase, black lowercase
    w_pcs = [piece for piece in "".join(board) if piece.isupper()]
    b_pcs = [piece for piece in "".join(board) if piece.islower()]
    #count total pieces (including king), and also count just the pawns
    ttl_white_pcs = len(w_pcs)
    ttl_black_pcs = len(b_pcs)
    ttl_white_pwns = len([p for p in w_pcs if p == "P"])
    ttl_black_pwns = len([p for p in b_pcs if p == "p"])

    #create material counts
    def sum_piece_values(pcs):
        """
        Accepts as input b_pcs or w_bcs and outputs the total value of what's given.
        """
        # dictionary for piece values
        chess_piece_values = {
            'P': 1,  # Pawn
            'N': 3,  # Knight
            'B': 3,  # Bishop
            'R': 5,  # Rook
            'Q': 9,  # Queen
            'K': 0,   # King (often considered to have infinite value, but in practical terms, it is 0 for valuation purposes)
        }
        count = 0
        for i in pcs:
            count += chess_piece_values[i.upper()]
        return count
    
    w_material_count=sum_piece_values(w_pcs)
    b_material_count=sum_piece_values(b_pcs)
        
    if game.winner == "white":
        win_pcs, lose_pcs = w_pcs, b_pcs
        winner_ttl_pieces_count = ttl_white_pcs
        loser_ttl_pieces_count = ttl_black_pcs
        winner_pawns_count = ttl_white_pwns
        loser_pawns_count = ttl_black_pwns
        winner_material_ttl = w_material_count
        loser_material_ttl = b_material_count        
        
    else: #includes DRAWs
        win_pcs, lose_pcs = b_pcs, w_pcs
        winner_ttl_pieces_count = ttl_black_pcs
        loser_ttl_pieces_count = ttl_white_pcs
        winner_pawns_count = ttl_black_pwns
        loser_pawns_count = ttl_white_pwns
        winner_material_ttl = b_material_count
        loser_material_ttl = w_material_count
    
    #count of just the major/minor pieces, -1 to not include King,    
    winner_pieces_only_count = winner_ttl_pieces_count - winner_pawns_count -1
    loser_pieces_only_count = loser_ttl_pieces_count - loser_pawns_count -1
    
    output = {
        #pieces on the board including king
        "winner_ttl_pieces_count" : winner_ttl_pieces_count, 
        "loser_ttl_pieces_count" : loser_ttl_pieces_count,
#       #count pawns
        "winner_pawns_count" : winner_pawns_count, 
        "loser_pawns_count" : loser_pawns_count,
        #count of just the major/minor pieces
        "winner_pieces_only_count" : winner_pieces_only_count,
        "loser_pieces_only_count" : loser_pieces_only_count,
        "pieces_count_diff" : winner_pieces_only_count - loser_pieces_only_count,
        "pawns_count_diff" : winner_pawns_count - loser_pawns_count,
        #count the material worth
        "winner_material_ttl" : winner_material_ttl,
        "loser_material_ttl" : loser_material_ttl,
        #might as well throw the pieces in too
        "winner_pcs" : win_pcs,
        "loser_pcs"  : lose_pcs,
    }
    
    return output

test_game = all_games_df_raw.query("time_class == 'bullet'").iloc[9]
# print(test_game.final_position_fen)
pieces_final_position(test_game).keys()

In [ ]:
def add_new_columns(games_df_in, printing=True):
    
    orig_cols = games_df_in.columns
    
    #uses winner_rating function to calculate the winner and rating swing
    win_rating_cols = games_df_in.apply(process_winner_rating, axis="columns", result_type="expand")
    
    #uses process moves function to break down the moves into w/b, and when castled
    move_cols =       games_df_in.apply(process_moves, axis="columns", result_type="expand")
    
    #add the above columns for now, so that the below funcs know who the winner was
    games_df_win = pd.concat([win_rating_cols, move_cols, games_df_in], axis=1)

    #drop duplicated columns
    #https://stackoverflow.com/questions/32041245/fast-method-for-removing-duplicate-columns-in-pandas-dataframe
    unique_cols = ~games_df_win.columns.duplicated()
    games_df_win = games_df_win.loc[:, unique_cols]
    
    display(games_df_win.columns)
    #create final position columns, gets the material left on the board
    final_pos_cols = games_df_win.apply(pieces_final_position, axis="columns", result_type="expand")
    
    #now calculate castle cols using some of the above columns
    cstl_cols =       games_df_win.apply(user_castled, axis="columns", result_type="expand")
    games_df_out = pd.concat([games_df_win, cstl_cols, final_pos_cols], axis=1)
    
    if printing:
        print("Created win/rating columns:\n" + "\n".join([f"- {col}" for col in win_rating_cols]))
        print("Created moves columns:\n" + "\n".join([f"- {col}" for col in move_cols]))
        print("Created final position columns:\n" + "\n".join([f"- {col}" for col in final_pos_cols]))
        print("Created castling columns:\n" + "\n".join([f"- {col}" for col in cstl_cols]))
        print("\nStarted with:\n" + "\n".join([f"- {col}" for col in orig_cols]))
    return games_df_out
    
all_games_df = add_new_columns(all_games_df_raw)
all_games_df.sample(2)

In [ ]:
all_games_df.columns

In [ ]:
test_game = all_games_df.query("time_class == 'bullet'").iloc[10]
test_game

## order the columns

In [ ]:
all_cols_order = [
    #meta
    'date', 'start_time', 'end_time',
    'time_class', 'time_control', 'rated', 'rules',
    'url', 'moves',
    'opening_code', 'opening_name',
    'white_username', 'black_username',
    'white_clock', 'black_clock',
    
    #ratings/results
    'white_rating', 'black_rating',
    'result', 'white_result', 'black_result',
    'higher_rated_colour', 'winner', 
    'winner_rating_diff', 'user_rating_diff',
    'user_win', 'user_colour', 
    'user_rating', 'move_numbers',
     
    #moves
    'white_moves',  'black_moves',
    'w_cstl_side', 'w_cstl_move', 
    'b_cstl_side', 'b_cstl_move',
    'user_castled', 'opp_castled',
    
    #final position
     'final_position_fen',
    'winner_ttl_pieces_count','loser_ttl_pieces_count',
    'winner_pawns_count', 'loser_pawns_count',
    'winner_pieces_only_count', 'loser_pieces_only_count',
    'pieces_count_diff', 'pawns_count_diff',
    'winner_material_ttl', 'loser_material_ttl',
    'winner_pcs', 'loser_pcs'
]

cols_pre = all_games_df.columns
all_games_df = all_games_df[all_cols_order]

removed_cols = [col for col in all_games_df.columns if col not in cols_pre]
if removed_cols:
    print("You have excluded the following columns:\n" + "\n".join([f"- {col}" for col in removed_cols]))

## Save to file

In [ ]:
# filename = "all_games_all_cols.csv"
# all_games_df.to_csv(os.path.join("..","data",filename), index=False)

# Selecting data

In [ ]:
def exclude_games(df, diff_max = 400):
    #if True, remove corresponding games
    constraints = {
        "chess960" : True,
        "daily" : True,
        "unrated" : True,
        "rating": True,
        "drawn" : False
    }
    
    queries = {
        #"title" : [ False_query, True_query] -> [to remove, to keep]
        "chess960" : ["rules != 'chess'", "rules == 'chess'"],
        "unrated" : ["rated != True", "rated == True"],
        "daily" : ["time_class == 'daily'", "time_class != 'daily'"],
        "rating" : [f"abs(winner_rating_diff) > {diff_max}", f"abs(winner_rating_diff) <= {diff_max}"],
        "drawn" :["winner == 'DRAW'", "winner != 'DRAW'"]
    }
    
    selected_games_df = df.copy()
    
    for constraint, exclude in constraints.items():
        exclude_df = df.query(queries[constraint][0])            
        include_df = selected_games_df.query(queries[constraint][1])
        suffix = "include"
        
        if exclude:
            selected_games_df = include_df
            suffix="exclude"
            
        if constraint != "rating":
            print(f"Found {len(exclude_df)} {constraint} games to {suffix}")
        else:
            print(f"Found {len(exclude_df)} games with a rating difference over {diff_max} to {suffix}")

    print(f"\nExcluded a total of {len(df) - len(selected_games_df)} rows due to constraints")
    print(f"Leaving {len(selected_games_df)} games to work with")
    
    return selected_games_df
    
exclude_games(all_games_df)
#test not excluding, test rating change

### Exclude rows

In [ ]:
selected_games_df = all_games_df.copy()

#exclude any chess960 games
selected_games_df = all_games_df.query("rules == 'chess'")

#exclude unrated games
selected_games_df = selected_games_df.query("rated == True")

#exclude daily games
selected_games_df = selected_games_df.query("time_class != 'daily'")

#exclude excess rating diff
diff_max = 400
selected_games_df = selected_games_df.query(f"abs(winner_rating_diff) <= {diff_max}")

print(f"Excluded {len(all_games_df) - len(selected_games_df)} rows due to constraints")
print(f"Leaves {len(selected_games_df)} games to work with")

draws_df = selected_games_df.query("winner=='DRAW'") #result=='1/2-1/2'
print(f"Of which {len(draws_df)} are draws")

selected_games_df.sample(3)

### Select columns

In [ ]:
select_cols = [
        #metadata
        'date', 'url',
        'start_time', 'end_time', 
        'time_class', 'time_control',
#         'rated', 'rules', #used to filter out unwanted games
        'white_username', 'black_username', "user_colour",
        #ratings/result
        'result', 'winner', 'higher_rated_colour',
        'white_rating', 'black_rating', 
        'white_result', 'black_result',
        'winner_rating_diff', "user_rating_diff", "user_rating",
        #specific moves of the game
        'opening_name', 'opening_code',
        'w_cstl_side', 'w_cstl_move', 
        'b_cstl_side', 'b_cstl_move',
        'user_castled', 'opp_castled',
        #final position stats
        'winner_ttl_pieces_count', 'loser_ttl_pieces_count',
        'winner_pawns_count', 'loser_pawns_count',
        'winner_pieces_only_count', 'loser_pieces_only_count',
        'pieces_count_diff', 'pawns_count_diff', 
        'winner_material_ttl', 'loser_material_ttl',
        'winner_pcs', 'loser_pcs',
        #target
        'user_win' 
]

selected_games_df = selected_games_df[select_cols]

removed_cols = [col for col in all_games_df.columns if col not in selected_games_df.columns]
print("You have excluded the following columns:\n" + "\n".join([f"- {col}" for col in removed_cols]))



## Inspect selected data

In [ ]:
selected_games_df.sample(5)

## Save selected data

In [ ]:
dates = [part for part in filename.split('_') if '-' in part]
print(dates) 

In [ ]:
filename_select = f"select_games_{dates[0]}_to_{dates[1]}_select_cols.csv"
selected_games_df.to_csv(os.path.join("..","data",filename), index=False)